<a href="https://colab.research.google.com/github/vadim-privalov/Sorevnovania/blob/main/%D0%92%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D0%B5_%D1%80%D1%8F%D0%B4%D1%8B_%D0%B0%D0%BF%D1%80%D0%B5%D0%BB%D1%8C_2022_%D0%B2%D0%B0%D1%80%D0%B8%D0%B0%D0%BD%D1%82_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt #Отрисовка изображений
#Отрисовывать изображения в ноутбуке, а не в консоль или файл
%matplotlib inline

In [ ]:
pip install --upgrade category_encoders

     |████████████████████████████████| 86 kB 3.0 MB/s 


In [ ]:
pip install catboost

     |████████████████████████████████| 76.6 MB 1.6 MB/s 


In [ ]:
# Basic Libraries
from IPython.display import clear_output
import os
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling as pp
import seaborn as sns
from IPython.display import display

# Encoders
from pandas.api.types import CategoricalDtype
from category_encoders import MEstimateEncoder
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler
from category_encoders import MEstimateEncoder

# Scikit Learn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, learning_curve, train_test_split, GroupKFold

# Algorithms
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Optuna - Bayesian Optimization 
#import optuna
#from optuna.samplers import TPESampler

# Plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go

# TPS April22 Metric
from sklearn.metrics import roc_auc_score

# Permutation Importance
#import eli5
#from eli5.sklearn import PermutationImportance

# TensorFlow
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, LSTM, GRU
from tensorflow.keras.layers import Bidirectional, Multiply

warnings.filterwarnings('ignore')

def load_data():
    df_train = pd.read_csv(filepath_or_buffer = "/content/drive/MyDrive/my_databases/tabular-playground-series-apr-2022/train.csv",
                           sep = ',')
    df_test = pd.read_csv(filepath_or_buffer = "/content/drive/MyDrive/my_databases/tabular-playground-series-apr-2022/test.csv",
                           sep = ',')
    train_labels = pd.read_csv(filepath_or_buffer = "/content/drive/MyDrive/my_databases/tabular-playground-series-apr-2022/train_labels.csv",
                           sep = ',')
    # Merge the splits so we can process them together
    df_train = df_train.merge(train_labels, on='sequence', how='outer')
    df = pd.concat([df_train, df_test])
    return df

def plot_feature_importance(importance,names,model_type):
    
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_df = fi_df[fi_df.feature_importance > 0]
    fig = px.bar(fi_df, x='feature_names', y='feature_importance', color="feature_importance",
             color_continuous_scale='Blugrn')
    
    # General Styling
    fig.update_layout(height=750, bargap=0.2,
                  margin=dict(b=50,r=30,l=100,t=100),
                  title = "<span style='font-size:36px; font-family:Times New Roman'>Feature Importance Analysis</span>",                  
                  plot_bgcolor='rgb(242,242,242)',
                  paper_bgcolor = 'rgb(242,242,242)',
                  font=dict(family="Times New Roman", size= 14),
                  hoverlabel=dict(font_color="floralwhite"),
                  showlegend=False)
    fig.show()

df_data = load_data()
clear_output()
#pp.ProfileReport(df_data)

Анализируя предыдущий отчет о профилировании, мы смогли заметить следующее в отношении предоставленного набора данных:

2.3M приведенных примеров
Все переменные являются числовыми.
Никаких пропущенных значений.
Высокая корреляция между некоторыми парами характеристик датчиков.
Ни одна из функций датчика не распределена равномерно.
Мы собираемся провести более исчерпывающий анализ в следующем разделе.

Как мы можем наблюдать в предыдущем отчете из набора данных, у нас есть объем более 2,3 миллионов строк. В связи с этим, чтобы не возникало проблем с памятью в ядре, мы собираемся уменьшить его использование памяти с помощью следующей функции. Ниже мы можем оценить, что сокращение было успешным, поскольку нам удалось добиться сокращения на 50,7%.

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
 
    return df

df_data = reduce_mem_usage(df_data)

Mem. usage decreased to 155.14 Mb (50.7% reduction)


Хотя у нас был один в верхнем отчете, мы собираемся построить другой, чтобы проанализировать и изучить лучшие корреляции между нашими функциями. Для тех, кто заинтересован, мои визуализации будут сделаны с помощью библиотеки Plotly. Надеюсь, они вам понравятся! Не стесняйтесь писать мне в комментариях с любыми советами, чтобы улучшить их.

In [ ]:
fig = px.imshow(df_data[df_data.state.isnull() == False].corr(), color_continuous_scale='RdBu_r', color_continuous_midpoint=0, origin='lower', aspect='auto')
fig.update_xaxes(showgrid = False, linecolor='gray', linewidth = 2, zeroline = False)
fig.update_yaxes(showgrid = True, gridcolor='gray',gridwidth=0.5, linecolor='gray',linewidth=2, zeroline = False)

# General Styling
fig.update_layout(height=750, bargap=0.2,
                  margin=dict(b=50,r=30,l=100, t=100),
                  title = "<span style='font-size:36px; font-family:Times New Roman'>Heatmap</span>",                  
                  plot_bgcolor='rgb(242,242,242)',
                  paper_bgcolor = 'rgb(242,242,242)',
                  font=dict(family="Times New Roman", size= 14),
                  hoverlabel=dict(font_color="floralwhite"),
                  showlegend=False)
fig.show()

Интерпретация: мы можем сделать следующие выводы после анализа тепловой карты, построенной:

Функция состояния значительно слабо коррелирует практически с каждой функцией датчика. Датчик 2 - это тот, который наиболее коррелирует с целью.
Все корреляции положительны, за исключением нескольких из них.
Следующие характеристики датчиков коррелируют друг с другом: Датчик 0, Датчик 6, Датчик 9, Датчик 3, Датчик 7 и Датчик 11

У нас есть 38186 различных последовательностей (также с учетом тестовых последовательностей). Каждая последовательность состоит из 60 временных шагов. Поскольку обучаемые и испытуемые разные, мы не можем использовать объект в качестве функции. Наоборот: мы должны убедиться, что наш классификатор обобщается на ранее невидимые предметы. В следующей ячейке записной книжки мы проверяем, как часто каждый предмет встречается в обучающих данных. Самая редкая тема встречается только дважды; самая частая тема встречается в сто раз чаще; она встречается 199 раз. Картина для тестовых данных аналогична.

In [ ]:
subject_count = pd.DataFrame(df_data[df_data.state.isnull() == False].subject.value_counts().sort_values() // 60).reset_index()
subject_count.columns = ['subject','count']
subject_count['subject'] = range(1,673)

# chart
fig = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5], vertical_spacing=0.1, horizontal_spacing=0.1,
                    subplot_titles=('Sequence Count Distribution over Training Subjects','Sequence Count Distribution over Test Subjects'))

fig.add_trace(go.Bar(x=subject_count['subject'], y=subject_count['count'], marker = dict(color=px.colors.sequential.Viridis[5])), row = 1, col = 1)

fig.update_xaxes(showgrid = False, linecolor='gray', linewidth = 2, zeroline = False, row=1, col=1)
fig.update_yaxes(showgrid = False, linecolor='gray',linewidth=2, zeroline = False, row=1, col=1)

subject_count = pd.DataFrame(df_data[df_data.state.isnull() == True].subject.value_counts().sort_values() // 60).reset_index()
subject_count.columns = ['subject','count']
subject_count['subject'] = range(319)

fig.add_trace(go.Bar(x=subject_count['subject'], y=subject_count['count'], marker = dict(color=px.colors.sequential.Viridis[5])), row = 1, col = 2)

fig.update_xaxes(showgrid = False, linecolor='gray', linewidth = 2, zeroline = False, row=1, col=2)
fig.update_yaxes(showgrid = False, linecolor='gray',linewidth=2, zeroline = False, row=1, col=2)

# General Styling
fig.update_layout(height=400, bargap=0.2,
                  margin=dict(b=50,r=30,l=100),
                  title = "<span style='font-size:36px; font-family:Times New Roman'>Sequence Count Analysis</span>",                  
                  plot_bgcolor='rgb(242,242,242)',
                  paper_bgcolor = 'rgb(242,242,242)',
                  font=dict(family="Times New Roman", size= 14),
                  hoverlabel=dict(font_color="floralwhite"),
                  showlegend=False)

In [ ]:
mean_state = df_data[df_data.state.isnull() == False].groupby('subject').agg({'state':'mean'}).reset_index().sort_values(by='state',ascending=False)
fig = px.scatter(mean_state, x='subject',y='state', color="state", color_continuous_scale='Blugrn', size='state')

fig.update_xaxes(showgrid = False, showline = True, gridwidth = 0.05, linecolor = 'gray', zeroline = False, linewidth = 2)
fig.update_yaxes(showline = True, gridwidth = 0.05, linecolor = 'gray', linewidth = 2, zeroline = False)

# General Styling
fig.update_layout(height=500,
              margin=dict(b=50,r=30,l=100,t=100),
              title = "<span style='font-size:36px; font-family:Times New Roman'>Subject Probability State Analysis</span>",                  
              plot_bgcolor='rgb(242,242,242)',
              paper_bgcolor = 'rgb(242,242,242)',
              font=dict(family="Times New Roman", size= 14),
              hoverlabel=dict(font_color="floralwhite"),
              showlegend=False)
fig.show()

2.2.3 | Временные шаги
Функции временного шага позволяют нам моделировать зависимость от времени. Ряд зависит от времени, если его значения могут быть предсказаны с момента их возникновения. В серии продаж в твердом переплете мы можем предсказать, что продажи в конце месяца, как правило, выше, чем продажи в начале месяца.

In [ ]:
sensors = df_data[df_data.state.isnull() == False].columns[df_data[df_data.state.isnull() == False].columns.str.contains('sensor')]

steps_0 = df_data[df_data.state.isnull() == False][df_data[df_data.state.isnull() == False]['state']==0].pivot_table(
    index=['step'],
    values=sensors,
    aggfunc='mean'
)

steps_1 = df_data[df_data.state.isnull() == False][df_data[df_data.state.isnull() == False]['state']==1].pivot_table(
    index=['step'],
    values=sensors,
    aggfunc='mean'
)

def plot_steps0(fig, feature):
    fig.add_trace(go.Scatter(x=steps_0.index, y=steps_0[feature], 
                  mode='lines+markers', marker = dict(color = px.colors.qualitative.G10[9]), name='State 0', visible=(feature==default_sensor)))

    fig.update_xaxes(showgrid = False, linecolor='gray', linewidth = 2, zeroline = False)
    fig.update_yaxes(showgrid = False, linecolor='gray', linewidth=2, zeroline = False)
    
def plot_steps1(fig, feature):
    fig.add_trace(go.Scatter(x=steps_1.index, y=steps_1[feature], 
                  mode='lines+markers', marker = dict(color = px.colors.sequential.Viridis[6]), name='State 1', visible=(feature==default_sensor)))
    
    fig.update_xaxes(showgrid = False, linecolor='gray', linewidth = 2, zeroline = False)
    fig.update_yaxes(showgrid = False, linecolor='gray', linewidth=2, zeroline = False)

fig = go.Figure()

sensor_plot_names = []
buttons=[]
default_sensor = "sensor_00"

for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"
    plot_steps0(fig, sensor_name)
    plot_steps1(fig, sensor_name)
    
    sensor_plot_names.extend([sensor_name]*2)
    
for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"
    buttons.append(dict(method='update',
                        label=sensor_name,
                        args = [{'visible': [sensor_name==r for r in sensor_plot_names]}]))

# General Layout. Add dropdown menus to the figure
fig.update_layout(height=500,
              margin=dict(b=50,r=30,l=100,t=100),
              title = "<span style='font-size:36px; font-family:Times New Roman'>Time Steps Analysis</span>",                  
              plot_bgcolor='rgb(242,242,242)',
              paper_bgcolor = 'rgb(242,242,242)',
              font=dict(family="Times New Roman", size= 14),
              hoverlabel=dict(font_color="floralwhite"),
              showlegend=True, updatemenus=[{"buttons": buttons, "direction": "down", "active": 0, "showactive": True, "x": 0.5, "y": 1.22}])

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="top",
    y=1.21,
    xanchor="right",
    x=1,
    #bgcolor="white",
    #bordercolor="Black",
    font=dict(
        family="Times New Roman",
        size=12
    #    color="black"
    ),
))

fig.show()

📌 Интерпретация: на этой предыдущей диаграмме мы нанесли на график среднее значение каждого датчика за шаг, в зависимости от того, равно ли состояние 0 или 1. Мы можем оценить, что для большинства датчиков графики имеют пики и впадины. Однако давайте взглянем на случай датчика 2. Удивительно, но обе линии рассеяния не обрываются ни в одной точке. Действительно, обе строки кажутся почти постоянными функциями с очень разными значениями.

2.3.2 | Дистрибутивы

In [ ]:
figure = plt.figure(figsize=(16, 8))
for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"
    plt.subplot(4, 4, sensor+1)
    plt.hist(df_data[df_data.state.isnull() == False][f"{sensor_name}"], bins=100)
    plt.title(f"{sensor_name}")
figure.tight_layout(h_pad=1.0, w_pad=0.5)
plt.suptitle('Sensor Normal Probability Plot', y=1.02)
plt.show()

Интерпретация: мы можем сделать следующие выводы после анализа построенных гистограмм:

Гистограммы показывают, что у каждого датчика есть выбросы.
Большинство значений характеристик датчиков содержат большое количество нулей.
Функции датчиков обычно не распределяются.
2.3.2 | Асимметрия и Эксцесс
Учебное пособие по асимметрии и эксцессу

Давайте теперь проверим асимметрию. Асимметрия - это мера асимметрии распределения вероятностей вещественной случайной величины относительно ее среднего значения. Значение асимметрии может быть положительным, нулевым, отрицательным или неопределенным.

Для унимодального распределения отрицательный перекос обычно указывает на то, что хвост находится в левой части распределения, а положительный перекос указывает на то, что хвост находится справа. В тех случаях, когда один хвост длинный, а другой толстый, асимметрия не подчиняется простому правилу. Например, нулевое значение означает, что хвосты по обе стороны от среднего значения в целом уравновешиваются; это относится к симметричному распределению, но также может быть верно для асимметричного распределения, где один хвост длинный и тонкий, а другой короткий, но толстый. Асимметрия, превышающая 1, обычно считается асимметричной, поэтому проверяйте в основном те, которые больше 1.

In [ ]:
from scipy.stats import skew, kurtosis
sensors = quantitative = [f for f in df_data.columns if df_data.dtypes[f] == 'float32']
sensors.remove('state')
skew_features = pd.DataFrame(df_data[sensors].apply(lambda x : skew(x))).reset_index()
skew_features.columns = ['sensor','skewness']
line = pd.DataFrame(dict(
    x = ['sensor_00','sensor_12'],
    y = [1, 1]
))

fig = px.bar(skew_features, x='sensor',y='skewness', color = 'skewness', color_continuous_scale = 'Blugrn', range_y=[-14,10])

fig.update_xaxes(showgrid = False, showline = True, gridwidth = 0.05, linecolor = 'gray', linewidth = 2)
fig.update_yaxes(showline = True, gridwidth = 0.05, linecolor = 'gray', linewidth = 2, zerolinecolor = 'gray', zerolinewidth = 0.5)

# General Styling
fig.update_layout(height=500,
              margin=dict(b=50,r=30,l=100,t=100),
              title = "<span style='font-size:36px; font-family:Times New Roman'>Sensors' Skewness</span>",                  
              plot_bgcolor='rgb(242,242,242)',
              paper_bgcolor = 'rgb(242,242,242)',
              font=dict(family="Times New Roman", size= 14),
              hoverlabel=dict(font_color="floralwhite"),
              showlegend=False)
fig.show()


Давайте теперь перейдем к эксцессу. В теории вероятностей и статистике эксцесс - это мера "хвостовости" распределения вероятностей вещественной случайной величины. Как и асимметрия, эксцесс описывает форму распределения вероятностей, и существуют различные способы его количественной оценки для теоретического распределения и соответствующие способы его оценки по выборке из совокупности. Различные показатели эксцесса могут иметь разную интерпретацию.

Стандартная мера эксцесса распределения, разработанная Карлом Пирсоном, представляет собой масштабированную версию четвертого момента распределения. Это число связано с хвостами распределения, а не с его пиком; следовательно, иногда встречающаяся характеристика эксцесса как "пиковости" неверна. Для этого показателя более высокий эксцесс соответствует большей экстремальности отклонений (или выбросов), а не конфигурации данных вблизи среднего значения.

In [ ]:
skew_features = pd.DataFrame(df_data[sensors].apply(lambda x : kurtosis(x))).reset_index()
skew_features.columns = ['sensor','kurtosis']
fig = px.bar(skew_features, x='sensor',y='kurtosis', color = 'kurtosis', color_continuous_scale = 'Blugrn')

fig.update_xaxes(showgrid = False, showline = True, gridwidth = 0.05, linecolor = 'gray', linewidth = 2)
fig.update_yaxes(showline = True, gridwidth = 0.05, linecolor = 'gray', linewidth = 2, zerolinecolor = 'gray', zerolinewidth = 0.5)

# General Styling
fig.update_layout(height=500,
              margin=dict(b=50,r=30,l=100,t=100),
              title = "<span style='font-size:36px; font-family:Times New Roman'>Sensors' Kurtosis</span>",                  
              plot_bgcolor='rgb(242,242,242)',
              paper_bgcolor = 'rgb(242,242,242)',
              font=dict(family="Times New Roman", size= 14),
              hoverlabel=dict(font_color="floralwhite"),
              showlegend=False)
fig.show()

Теперь давайте построим "внутреннюю" часть гистограммы после удаления 2 % выбросов с обеих сторон. Мы видим, что датчики отличаются по своим характеристикам:

Большинство измерений sensor_02 возвращают значения, кратные 0,33 (но промежуточные значения встречаются с низкой вероятностью).
Sensor_08 имеет дискретные значения (кратные 0,1).
Некоторые датчики выглядят как обычные распределения, другие (в частности, sensor_12) имеют длинный хвост.
Распределения выглядят симметричными с центром в 0. Эта следующая ячейка EDA была взята из раздела Ambrose EDU.

In [ ]:
figure = plt.figure(figsize=(16, 8))
for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"
    plt.subplot(4, 4, sensor+1)
    plt.hist(df_data[df_data.state.isnull() == False][sensor_name], bins=100,
             range=(df_data[df_data.state.isnull() == False][sensor_name].quantile(0.02),
                    df_data[df_data.state.isnull() == False][sensor_name].quantile(0.98)))
    plt.title(f"{sensor_name} histogram")
figure.tight_layout(h_pad=1.0, w_pad=0.5)
plt.suptitle('Sensor Histograms After Outlier Removal', y=1.02)
plt.show()

2.3.3 График Нормальной Вероятности
График нормальной вероятности - это графический метод для оценки того, является ли набор данных приблизительно нормально распределенным. Данные сопоставляются с теоретическим нормальным распределением таким образом, чтобы точки образовывали приблизительную прямую линию.

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from scipy import stats
figure = plt.figure(figsize=(14, 8))
for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"
    plt.subplot(4, 4, sensor+1)
    stats.probplot(df_data[df_data.state.isnull() == False][f"{sensor_name}"], plot=plt)
    plt.title(f"{sensor_name}")
figure.tight_layout(h_pad=1.0, w_pad=0.5)
plt.suptitle('Sensor Normal Probability Plot', y=1.02)
plt.show()

2.4.1 | Определение выбросов
Выброс - это наблюдение, которое численно отличается от остальных данных, или, проще говоря, это значение, которое находится вне диапазона.давайте возьмем пример, чтобы проверить, что происходит с набором данных с выбросами и набором данных без них.

2.4.2 | Обнаружение выбросов
Выбросы могут быть двух типов: Одномерные и многомерные. Эти выбросы можно обнаружить, когда мы смотрим на распределение одной переменной. Многомерные выбросы - это выбросы в n-мерном пространстве. Мы начнем с определения того, есть ли выбросы в нашем наборе данных или нет.

In [ ]:
import scipy.stats as stats
def grubbs_test(x):
    n = len(x)
    mean_x = np.mean(x)
    sd_x = np.std(x)
    numerator = max(abs(x-mean_x))
    g_calculated = numerator/sd_x
    print("Grubbs Calculated Value:",g_calculated)
    t_value = stats.t.ppf(1 - 0.05 / (2 * n), n - 2)
    g_critical = ((n - 1) * np.sqrt(np.square(t_value))) / (np.sqrt(n) * np.sqrt(n - 2 + np.square(t_value)))
    print("Grubbs Critical Value:",g_critical)
    if g_critical > g_calculated:
        print("From grubbs_test we observe that calculated value is lesser than critical value, Accept null hypothesis and conclude that there is no outliers\n")
    else:
        print("From grubbs_test we observe that calculated value is greater than critical value, Reject null hypothesis and conclude that there is an outliers\n")

2.4.2.2 | Метод Z-оценки
Используя метод Z score, мы можем выяснить, на сколько стандартных отклонений значение отклоняется от среднего.

In [ ]:
out=[]
def Zscore_outlier(df):
    m = np.mean(df)
    sd = np.std(df)
    row = 0
    for i in df: 
        z = (i-m)/sd
        if np.abs(z) > 3: 
            out.append(row)
        row += 1
    return out

2.4.3 | Обнаружение выбросов Датчиков

In [ ]:
def plot_outliers(feature):
    grubbs_test(df_data[df_data.state.isnull() == False][feature])
    outliers_index = Zscore_outlier(df_data[df_data.state.isnull() == False][feature])
    print('Outlier %: ', len(outliers_index)/df_data[df_data.state.isnull() == False].shape[0])
    print('\n')

    df_outlier = df_data[df_data.state.isnull() == False]
    df_outlier['outlier'] = 0
    df_outlier.loc[outliers_index,'outlier'] = 1

    fig = px.scatter(df_outlier.loc[outliers_index,:], x=outliers_index, y=feature, color = feature, color_continuous_scale='Blugrn', color_continuous_midpoint=0)
    fig.update_xaxes(showgrid = False, showline = True, gridwidth = 0.05, linecolor = 'gray', zeroline = False, linewidth = 2)
    fig.update_yaxes(showline = True, gridwidth = 0.05, linecolor = 'gray', linewidth = 2, zeroline = False)

    #General Styling
    fig.update_layout(height=400, bargap=0.2,
                      margin=dict(b=50,r=30,l=100, t=80),
                      title = "<span style='font-size:36px; font-family:Times New Roman'>Sensor Features Outliers Analysis</span>",                  
                      plot_bgcolor='rgb(242,242,242)',
                      paper_bgcolor = 'rgb(242,242,242)',
                      font=dict(family="Times New Roman", size= 14),
                      hoverlabel=dict(font_color="floralwhite"),
                      showlegend=False)
    fig.show()

sensors = quantitative = [f for f in df_data.columns if df_data.dtypes[f] == 'float32']
sensors.remove('state')
i = 0
for sensor in sensors:
    print('Sensor ', i)
    print('\n')
    i+=1
    plot_outliers(sensor)

3 | Разработка функциональных возможностей
3.1 | Взаимная информация

Взаимная информация описывает отношения в терминах неопределенности. Взаимная информация (MI) между двумя величинами является мерой степени, в которой знание одной величины уменьшает неопределенность в отношении другой. Если бы вы знали ценность функции, насколько увереннее вы были бы в отношении цели? Scikit-learn имеет две метрики взаимной информации в своем модуле feature_selection: одну для вещественных целей (mutual_info_regression) и одну для категориальных целей (mutual_info_classif). Следующая ячейка вычисляет оценки MI для наших функций и заключает их в красивый фрейм данных. В дальнейшем мы собираемся отказаться от неинформативных функций, поскольку они бесполезны.

In [ ]:
from sklearn.feature_selection import mutual_info_classif

def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object","category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    #discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_classif(X, y, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

def uninformative_cols(df, mi_scores):
    return df.loc[:, mi_scores == 0.0].columns
    
y = df_data[df_data.state.isnull() == False]['state']
x = df_data[df_data.state.isnull() == False].drop(['state','sequence','subject'],axis=1)

mi_scores = make_mi_scores(x, y)
#col = uninformative_cols(x, mi_scores)
#x = x.drop(col,axis=1)
print('Uninformative features:\n{}'.format(mi_scores[mi_scores == 0.0]))
mi_scores = pd.DataFrame(mi_scores).reset_index().rename(columns={'index':'Feature'})

Uninformative features:
sensor_05    0.0
Name: MI Scores, dtype: float64


Далее мы собираемся отобразить результаты, полученные ранее, чтобы определить, какие функции являются наиболее информативными, а какие требуют дополнительного анализа. Вы можете видеть, что у нас есть ряд функций, которые очень информативны, а также некоторые, которые вообще не кажутся информативными (по крайней мере, сами по себе). Функции с наивысшим рейтингом обычно приносят наибольшую отдачу во время разработки функций, поэтому было бы неплохо сосредоточить свои усилия на них. С другой стороны, обучение неинформативным функциям может привести к переобучению.

In [ ]:
fig = px.bar(mi_scores, x='MI Scores', y='Feature', color="MI Scores", color_continuous_scale='darkmint')

fig.update_xaxes(showgrid = False, showline = True, gridwidth = 0.05, linecolor = 'gray', zeroline = False, linewidth = 2)
fig.update_yaxes(showline = True, gridwidth = 0.05, linecolor = 'gray', linewidth = 2, zeroline = False)

fig.update_layout(height = 750, title_text="Mutual Information Scores", plot_bgcolor='rgb(242, 242, 242)', paper_bgcolor = 'rgb(242, 242, 242)',
                  title_font=dict(size=29, family="Lato, sans-serif"), xaxis={'categoryorder':'category ascending'}, margin=dict(t=80))

📌 Интерпретация: мы можем оценить, что каждая функция датчика имеет очень низкое значение взаимной информации. Это означает, что они не кажутся такими информативными, по крайней мере, сами по себе. Поэтому в следующих разделах мы сосредоточимся на создании функций, связанных с этими датчиками, чтобы получить функции с лучшими показателями MI.

3.2 | Функции статистики

В этом разделе мы создадим некоторые новые функции, связанные с сенсорными. Мы собираемся начать с некоторых статистических характеристик. Процедура будет группироваться по каждой отдельной последовательности, а затем добавляться функции для: максимального, минимального, среднего, медианного, std и эксцесса. Далее мы проведем исчерпывающий анализ, чтобы изучить, какие статистические функции работают лучше.

In [ ]:
import scipy.stats
sensors = [col for col in df_data.columns if 'sensor_' in col]
for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"    
    df_data[f'{sensor_name}''_max'] = df_data.groupby('sequence')[f'{sensor_name}'].transform('max') 
    df_data[f'{sensor_name}''_min'] = df_data.groupby('sequence')[f'{sensor_name}'].transform('min') 
    df_data[f'{sensor_name}''_mean'] = df_data.groupby('sequence')[f'{sensor_name}'].transform('mean')  
    df_data[f'{sensor_name}''_std'] = df_data.groupby('sequence')[f'{sensor_name}'].transform('std')  
    df_data[f'{sensor_name}''_median'] = df_data.groupby('sequence')[f'{sensor_name}'].transform('median')
    df_data[f'{sensor_name}''_flip'] = df_data[f'{sensor_name}'] * -1
# Kurtosis
sequence = df_data.sequence.unique()
length = range(60)
for sensor in range(13):
    kurtosis = []
    sensor_name = f"sensor_{sensor:02d}"
    for i in sequence: 
        kurt = scipy.stats.kurtosis(df_data[df_data.sequence == i][sensor_name])
        for j in length:
            kurtosis.append(kurt)
    df_data[sensor_name + '_kurtosis'] = kurtosis

KeyboardInterrupt: ignored

3.2.1 | Выбор функции
В этом разделе мы проанализируем полезность функций, которые мы только что создали. Мы построим график зависимости цели от каждой функции, то есть диаграмму 푃(푦=1|푥) . Чтобы получить осмысленный график, мы применяем два преобразования:ns:

Ось x - это не значение объекта, а его индекс (при сортировке по значению объекта).
Ось y - это не целевое значение (которое может быть только 0 или 1), а скользящее среднее значение по 1000 целям.
Объекты с горизонтальной линией в виде диаграммы (вероятность положительной цели равна 0,5 независимо от значения объекта) будут считаться плохими, а не полезными. С другой стороны, хорошие характеристики будут иметь кривую с высокой 𝑦𝑚𝑎𝑥−𝑦𝑚𝑖𝑛 .

3.2.1.1 | Максимальные характеристики
Мы можем заметить, что почти каждый из них представляет "нормальное" (недостаточно высокое, чтобы быть выбранным напрямую) значение разницы между максимальным и минимальным значением на их соответствующей диаграмме. Таким образом, они могли бы быть полезны для нас. Мы пока не отказываемся ни от одного из них.

In [ ]:
state = []
length = range(int(df_data[df_data.state.isnull() == False].shape[0] / 60))
df_train = df_data[df_data.state.isnull() == False]
for j in length:
    state.append(df_train.at[60*j,'state'])

max_df = pd.DataFrame({'state':state})
max_cols = [col for col in df_data.columns if '_max' in col]
for col in max_cols:
    values=[]
    for i in length:
         values.append(df_train.at[60*i,col])
    max_df[col] = values
            
plt.subplots(1, 4, sharey=True, sharex=True, figsize=(20, 25))
counter = 1
for col in max_df.drop('state',axis=1).columns:
    temp = pd.DataFrame({col: max_df[col].values,
                        'state': max_df['state']})
    
    temp = temp.sort_values(col)
    temp.reset_index(inplace=True)
    plt.subplot(15, 4, counter)
    plt.scatter(temp.index, temp.state.rolling(1000).mean(), s=2)
    plt.xlabel(col)
    plt.xticks([])
    counter = counter +1
plt.show()

3.2.1.2 | Минимальные характеристики
В этом случае мы можем заметить, что есть некоторые функции (например, sensor_06_min), которые представляют собой почти горизонтальную линейную диаграмму. Эти функции, как отмечалось ранее, будут не очень полезны.

In [ ]:
max_df = pd.DataFrame({'state':state})
max_cols = [col for col in df_data.columns if '_min' in col]
for col in max_cols:
    values=[]
    for i in length:
         values.append(df_train.at[60*i,col])
    max_df[col] = values
            
plt.subplots(1, 4, sharey=True, sharex=True, figsize=(20, 25))
counter = 1
for col in max_df.drop('state',axis=1).columns:
    temp = pd.DataFrame({col: max_df[col].values,
                        'state': max_df['state']})
    
    temp = temp.sort_values(col)
    temp.reset_index(inplace=True)
    plt.subplot(15, 4, counter)
    plt.scatter(temp.index, temp.state.rolling(1000).mean(), s=2)
    plt.xlabel(col)
    plt.xticks([])
    counter = counter +1
plt.show()

3.2.1.3 | Средние характеристики
Теперь, что касается диаграмм средних характеристик, мы можем заметить, что большая их часть представляет собой выпуклые кривые. Но, внимательно присмотревшись, мы можем заметить, что некоторые из них имеют низкое значение разницы между максимальным и минимальным значением. Некоторые из них: sensor_00, sensor_03, sensor_05, sensor_09, sensor_10.

In [ ]:
max_df = pd.DataFrame({'state':state})
max_cols = [col for col in df_data.columns if '_mean' in col]
for col in max_cols:
    values=[]
    for i in length:
         values.append(df_train.at[60*i,col])
    max_df[col] = values
            
plt.subplots(1, 4, sharey=True, sharex=True, figsize=(20, 25))
counter = 1
for col in max_df.drop('state',axis=1).columns:
    temp = pd.DataFrame({col: max_df[col].values,
                        'state': max_df['state']})
    
    temp = temp.sort_values(col)
    temp.reset_index(inplace=True)
    plt.subplot(15, 4, counter)
    plt.scatter(temp.index, temp.state.rolling(1000).mean(), s=2)
    plt.xlabel(col)
    plt.xticks([])
    counter = counter +1
plt.show()

3.2.1.4 | Средние характеристики
Медианные функции будут не очень полезны. Это те, у которых наименьшее значение разницы между максимальными и минимальными значениями.

In [ ]:
max_df = pd.DataFrame({'state':state})
max_cols = [col for col in df_data.columns if '_median' in col]
for col in max_cols:
    values=[]
    for i in length:
         values.append(df_train.at[60*i,col])
    max_df[col] = values
            
plt.subplots(1, 4, sharey=True, sharex=True, figsize=(20, 25))
counter = 1
for col in max_df.drop('state',axis=1).columns:
    temp = pd.DataFrame({col: max_df[col].values,
                        'state': max_df['state']})
    
    temp = temp.sort_values(col)
    temp.reset_index(inplace=True)
    plt.subplot(15, 4, counter)
    plt.scatter(temp.index, temp.state.rolling(1000).mean(), s=2)
    plt.xlabel(col)
    plt.xticks([])
    counter = counter +1
plt.show()

3.2.1.5 | Особенности Std
В этом случае мы можем наблюдать одну потенциальную особенность. Это случай sensor_02_std. Глядя на оси y, мы можем оценить, что значение разницы довольно велико.

In [ ]:
max_df = pd.DataFrame({'state':state})
max_cols = [col for col in df_data.columns if '_std' in col]
for col in max_cols:
    values=[]
    for i in length:
         values.append(df_train.at[60*i,col])
    max_df[col] = values
            
plt.subplots(1, 4, sharey=True, sharex=True, figsize=(20, 25))
counter = 1
for col in max_df.drop('state',axis=1).columns:
    temp = pd.DataFrame({col: max_df[col].values,
                        'state': max_df['state']})
    
    temp = temp.sort_values(col)
    temp.reset_index(inplace=True)
    plt.subplot(15, 4, counter)
    plt.scatter(temp.index, temp.state.rolling(1000).mean(), s=2)
    plt.xlabel(col)
    plt.xticks([])
    counter = counter +1
plt.show()

3.2.1.6 | Особенности эксцесса
Безусловно, функции эксцесса являются лучшими. Мы можем наблюдать множество потенциальных возможностей. Это, например, случай sensor_04_kurtosis. Глядя на оси y, мы можем оценить, что значение разницы невероятно велико.

In [ ]:
max_df = pd.DataFrame({'state':state})
max_cols = [col for col in df_data.columns if '_kurtosis' in col]
for col in max_cols:
    values=[]
    for i in length:
         values.append(df_train.at[60*i,col])
    max_df[col] = values
            
plt.subplots(1, 4, sharey=True, sharex=True, figsize=(20, 25))
counter = 1
for col in max_df.drop('state',axis=1).columns:
    temp = pd.DataFrame({col: max_df[col].values,
                        'state': max_df['state']})
    
    temp = temp.sort_values(col)
    temp.reset_index(inplace=True)
    plt.subplot(15, 4, counter)
    plt.scatter(temp.index, temp.state.rolling(1000).mean(), s=2)
    plt.xlabel(col)
    plt.xticks([])
    counter = counter +1
plt.show()

3.2.1.7 | Функции переворота
Мы можем заметить, что функции flip для нас совершенно бесполезны. Они получают на сегодняшний день наименьшее значение разницы между 𝑦𝑚𝑎𝑥−𝑦𝑚𝑖𝑛 .

In [ ]:
max_df = pd.DataFrame({'state':state})
max_cols = [col for col in df_data.columns if '_flip' in col]
for col in max_cols:
    values=[]
    for i in length:
         values.append(df_train.at[60*i,col])
    max_df[col] = values
            
plt.subplots(1, 4, sharey=True, sharex=True, figsize=(20, 25))
counter = 1
for col in max_df.drop('state',axis=1).columns:
    temp = pd.DataFrame({col: max_df[col].values,
                        'state': max_df['state']})
    
    temp = temp.sort_values(col)
    temp.reset_index(inplace=True)
    plt.subplot(15, 4, counter)
    plt.scatter(temp.index, temp.state.rolling(1000).mean(), s=2)
    plt.xlabel(col)
    plt.xticks([])
    counter = counter +1
plt.show()

📌 Интерпретация: мы можем сделать следующие выводы после анализа предыдущего графика, построенного:

Функции эксцесса, по-видимому, работают лучше, так как они получили самые высокие значения разницы между максимальными и минимальными значениями.
Функция стандартного отклонения для датчика 2 является одной из лучших.
Среднее и медианное значения кажутся бесполезными функциями.
Принимая во внимание все вышесказанное, мы собираемся отказаться от следующих функций:

In [ ]:
max_col = [col for col in df_data.columns if '_max' in col]
min_col = [col for col in df_data.columns if '_min' in col]
mean_col = [col for col in df_data.columns if '_mean' in col]
mean_col.remove('sensor_04_mean')
median_col = [col for col in df_data.columns if '_median' in col]
std_col = [col for col in df_data.columns if '_std' in col]
std_col.remove('sensor_02_std')
kurtosis_col = [col for col in df_data.columns if '_kurtosis' in col]
kurtosis_col.remove('sensor_10_kurtosis')
kurtosis_col.remove('sensor_04_kurtosis')
flip_col = [col for col in df_data.columns if '_flip' in col]

# Drop useless features
df_data = df_data.drop(max_col, axis=1)
df_data = df_data.drop(min_col, axis=1)
df_data = df_data.drop(mean_col, axis=1)
df_data = df_data.drop(std_col, axis=1)
df_data = df_data.drop(median_col, axis=1)
df_data = df_data.drop(kurtosis_col, axis=1)
df_data = df_data.drop(flip_col, axis=1)

3.3 | Особенности временных рядов

3.3.1 | Особенности задержки. Автокорреляция (ACF). Частичная автокорреляция (PACF)
Автокорреляция - Функция автокорреляции (ACF) измеряет, как ряд коррелирует сам с собой с различными задержками.
Частичная автокорреляция - функция частичной автокорреляции может быть интерпретирована как регрессия ряда относительно его прошлых лагов. Эти термины можно интерпретировать так же, как стандартную линейную регрессию, то есть вклад изменения в этом конкретном лаге при сохранении других постоянными.
Запаздывание временного ряда означает смещение его значений вперед на один или несколько временных шагов или, что эквивалентно, смещение времени в его индексе назад на один или несколько шагов. В любом случае эффект заключается в том, что наблюдения в запаздывающих рядах, по-видимому, произошли позже по времени. Здесь мы создали функцию задержки на 1 шаг, хотя возможен сдвиг на несколько шагов.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#plot_acf(df_data['sensor_02'],lags=2,title="Sensor 2")
#plot_pacf(df_data['sensor_03'],lags=2,title="Sensor 2")

In [ ]:
for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"    
    df_data[f"{sensor_name}" + '_lag1'] = df_data.groupby('sequence')[f"{sensor_name}"].shift(1)  
    df_data[f"{sensor_name}" + '_lag1'].fillna(0, inplace=True)
    df_data[f"{sensor_name}" + '_diff1'] = df_data[f"{sensor_name}"] - df_data[f"{sensor_name}" + '_lag1']
    #df_data[f"{sensor_name}" + '_flip'] = df_data[f"{sensor_name}"] * -1

4 | Modeling

In [ ]:
features = df_data.columns.tolist()[3:]
sc = StandardScaler()

train_df = df_data[df_data.state.isnull() == False]
test_df = df_data[df_data.state.isnull() == True]

train_df[features] = sc.fit_transform(train_df[features])
test_df[features] = sc.transform(test_df[features])

groups = train_df["sequence"]
train_labels = pd.read_csv('/content/drive/MyDrive/my_databases/tabular-playground-series-apr-2022/train_labels.csv')
labels = train_labels["state"]

train_df = train_df.drop(["sequence", "subject", "step",'state'], axis=1).values
train_df = train_df.reshape(-1, 60, train_df.shape[-1])

test_df = test_df.drop(["sequence", "subject", "step",'state'], axis=1).values
test_df = test_df.reshape(-1, 60, test_df.shape[-1])

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    BATCH_SIZE = tpu_strategy.num_replicas_in_sync * 64
    print("Running on TPU:", tpu.master())
    print(f"Batch Size: {BATCH_SIZE}")
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    BATCH_SIZE = 256
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    print(f"Batch Size: {BATCH_SIZE}")
    
def dnn_model():
    
    x_input = Input(shape=(train_df.shape[-2:]))
    
    x1 = Bidirectional(LSTM(units=512, return_sequences=True))(x_input)
    x2 = Bidirectional(LSTM(units=256, return_sequences=True))(x1)
    z1 = Bidirectional(GRU(units=256, return_sequences=True))(x1)
    
    c = Concatenate(axis=2)([x2, z1])
    
    x3 = Bidirectional(LSTM(units=128, return_sequences=True))(c)
    
    x4 = GlobalMaxPooling1D()(x3)
    x5 = Dense(units=128, activation='selu')(x4)
    x_output = Dense(1, activation='sigmoid')(x5)

    model = Model(inputs=x_input, outputs=x_output, name='lstm_model')
    
    return model

model = dnn_model()
clear_output()
plot_model(model, show_shapes=True)

In [ ]:
def plotHist(hist):
    plt.plot(hist.history["auc"])
    plt.plot(hist.history["val_auc"])
    plt.title("model performance")
    plt.ylabel("area_under_curve")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()
    return

In [ ]:
with tpu_strategy.scope():
    VERBOSE = True
    predictions, scores = [], []
    k = GroupKFold(n_splits = 10)

    for fold, (train_idx, val_idx) in enumerate(k.split(train_df, labels, groups.unique())):
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
    
        X_train, X_val = train_df[train_idx], train_df[val_idx]
        y_train, y_val = labels.iloc[train_idx].values, labels.iloc[val_idx].values
        
        model = dnn_model()
        model.compile(optimizer="adam", loss="binary_crossentropy", metrics='AUC')

        lr = ReduceLROnPlateau(monitor="val_auc", factor=0.6, 
                               patience=4, verbose=VERBOSE)

        es = EarlyStopping(monitor="val_auc", patience=2, 
                           verbose=VERBOSE, mode="max", 
                           restore_best_weights=True)
        
        save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
        chk_point = ModelCheckpoint(f'./TPS_model_2022_{fold+1}C.h5', options=save_locally, 
                                    monitor='val_auc', verbose=VERBOSE, 
                                    save_best_only=True, mode='max')
        
        history = model.fit(X_train, y_train, 
                  validation_data=(X_val, y_val), 
                  epochs=15,
                  verbose=VERBOSE,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
        model = load_model(f'./TPS_model_2022_{fold+1}C.h5', options=load_locally)
        
        y_pred = model.predict(X_val, batch_size=BATCH_SIZE).squeeze()
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)
        predictions.append(model.predict(test_df, batch_size=BATCH_SIZE).squeeze())
        print(f"Fold-{fold+1} | OOF Score: {score}")
        
        history_df = pd.DataFrame(history.history)
        history_df.loc[:, ['loss', 'val_loss']].plot();
        plt.title(f"{fold+1}")
        print("Minimum validation loss: {}".format(history_df['val_loss'].min()))
    
    print(f'Mean accuracy on {k.n_splits} folds - {np.mean(scores)}')

In [ ]:
ssub = pd.read_csv('/content/drive/MyDrive/my_databases/tabular-playground-series-apr-2022/sample_submission.csv')
ssub["state"] = sum(predictions)/k.n_splits 
ssub.to_csv('submission.csv', index=False)
ssub.head(3)